In [13]:
# データ取得
import pandas as pd
import numpy as np

df = pd.DataFrame([
['green', 'M',10.1,'class1','mon'],
['red','L',13.5,'class2','sun'],
['blue','XL',15.3,'class1','wed']])
df.columns = ['color' , 'size', 'price',
'classlabel','weekday']

In [14]:
df.head()

,color,size,price,classlabel,weekday
0,green,M,10.1,class1,mon
1,red,L,13.5,class2,sun
2,blue,XL,15.3,class1,wed


In [15]:
# one-hot表現
a = [3, 0, 8, 1, 9]
a_one_hot = np.identity(10)[a]
print(a_one_hot)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [16]:
# マッピング
# sizeを数値に置き換える
size_mapping = {'XL':3,'L':2,'M':1}
df['size'] = df['size'].map(size_mapping)
df


,color,size,price,classlabel,weekday
0,green,1,10.1,class1,mon
1,red,2,13.5,class2,sun
2,blue,3,15.3,class1,wed


In [17]:
# weekdayを数値に置き換える
daymap = {'sun':0,'mon':1,'tue':2,'wed':3,'thu':4,'fri':5,'sat':6}
df['weekday'] = df['weekday'].apply(lambda x :daymap[x])
df

,color,size,price,classlabel,weekday
0,green,1,10.1,class1,1
1,red,2,13.5,class2,0
2,blue,3,15.3,class1,3


In [18]:
# Label encoder
# colorを数値にする
from sklearn.preprocessing import LabelEncoder
X = df[['color', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 10.1],
       [2, 13.5],
       [0, 15.3]], dtype=object)

In [19]:
# 数値にしたcolorをone-hot表現にする
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 0. ,  1. ,  0. , 10.1],
       [ 0. ,  0. ,  1. , 13.5],
       [ 1. ,  0. ,  0. , 15.3]])

In [28]:
# Bags of words

from math import sqrt
import gensim
import gensim.parsing
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import re


def vec2dense(vec, num_terms):
    '''Convert from sparse gensim format to dense list of numbers'''
    return list(gensim.matutils.corpus2dense([vec], num_terms=num_terms).T[0])


if __name__ == '__main__':

    # Loding a corpus, remove the line break, convert to lower case
    docs = {}
    corpus_dir = 'corpus'
    for filename in os.listdir(corpus_dir):
        path = os.path.join(corpus_dir, filename)
        doc = open(path).read().strip().lower()
        docs[filename] = doc

    names = docs.keys()

    # Stopwords removal and Stemming
    print ("\n---Corpus with Stopwords Removed---")

    preprocessed_docs = {}
    for name in names:
        preprocessed = gensim.parsing.preprocess_string(docs[name])
        preprocessed_docs[name] = preprocessed
        print (name, ":", preprocessed)

    # Create a dictionary
    # except for a very high and low frequent word
    dct = gensim.corpora.Dictionary(preprocessed_docs.values())
    unfiltered = dct.token2id.keys()
    dct.filter_extremes(no_below=3, no_above=0.6)
    filtered = dct.token2id.keys()
    filtered_out = set(unfiltered) - set(filtered)

    print ("\nThe following super common/rare words are filtered out...")
    print (list(filtered_out), '\n')

    print ("Vocabulary after filtering...")
    print (dct.token2id.keys(), "(%d words)" % len(dct.token2id.keys()), '\n')

    print ("Save Dictionary...")
    dct_txt = "id2word.txt"
    dct.save_as_text(dct_txt)
    print ("  saved to %s\n" % dct_txt)

    # Bag of Words Vectors
    print ("---Bag of Words Corpus---")

    bow_docs = {}
    bow_docs_all_zeros = {}
    for name in names:

        sparse = dct.doc2bow(preprocessed_docs[name])
        bow_docs[name] = sparse
        dense = vec2dense(sparse, num_terms=len(dct))
        print (name, ":", dense)
        bow_docs_all_zeros[name] = all(d == 0 for d in dense)

    print ("\nall zeros...\n", [
        name for name in bow_docs_all_zeros if bow_docs_all_zeros[name]
    ])

    # Dimension reduction with LSI model
    print ("\n---LSI Model---")

    lsi_docs = {}
    num_topics = 2
    lsi_model = gensim.models.LsiModel(
        bow_docs.values(),
        id2word=dct.load_from_text('id2word.txt'),
        num_topics=num_topics)

    for name in names:

        vec = bow_docs[name]
        sparse = lsi_model[vec]
        dense = vec2dense(sparse, num_topics)
        lsi_docs[name] = sparse
        print (name, ":", dense)

    print ("\nTopics")
    print (lsi_model.print_topics())

    # Normalize a vector
    print ("\n---Unit Vectorization---")

    unit_vecs = {}
    for name in names:

        vec = vec2dense(lsi_docs[name], num_topics)
        norm = sqrt(sum(num**2 for num in vec))
        unit_vec = [num / norm for num in vec]
        unit_vecs[name] = unit_vec
        print (name, ":", unit_vec)

    # Binary classification
    print ("\n---Classification---\n")

    all_data = [unit_vecs[name] for name in names if re.match("ipad", name)]
    all_data.extend(
        [unit_vecs[name] for name in names if re.match("sochi", name)])

    all_labels = [1 for name in names if re.match("ipad", name)]
    all_labels.extend([2 for name in names if re.match("sochi", name)])

    train_data, test_data, train_label, test_label = train_test_split(
        all_data, all_labels)

    # Train SVM classifier
    classifier = SVC()
    classifier.fit(train_data, train_label)

    # Prediction
    predict_label = classifier.predict(test_data)

    target_names = ["class 'ipad'", "class 'sochi'"]
    print (classification_report(
        test_label, predict_label, target_names=target_names))

FileNotFoundError: [WinError 3] 指定されたパスが見つかりません。: 'corpus'

In [30]:
# Tf-idf

vectorizer = TfidfVectorizer(analyzer=utils.stems,min_df=1, max_df=50)
corpus = [page.text for page in pages]
x = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tfidfs = x.toarray()[constants.DOC_NUM]
print([term for term in terms if
is_bigger_than_min_tfidf(term, terms, tfidfs)])


NameError: name 'TfidfVectorizer' is not defined

In [35]:
# word2vec

from gensim.models import KeyedVectors
word1 = "猫"
word2 = "犬"
model = KeyedVectors.load_word2vec_format('entity_vector.model.bin',binary=True)
model.similarity(word1,word2)
model.most_similar(positive =[word1], negative = [],topn = 10)

FileNotFoundError: [Errno 2] No such file or directory: 'entity_vector.model.bin'